# UST Into the Wormhole
> A decentralised stablecoin into a decentralised bridge

- toc:true
- branch: master
- badges: true
- comments: false
- author: Scott Simpson
- categories: [Terra, Wormhole]
- hide: false  


In [1]:
#hide
#Imports & settings
!pip install plotly --upgrade
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline
%load_ext google.colab.data_table
%load_ext rpy2.ipython
%R options(tidyverse.quiet = TRUE)
%R options(lubridate.quiet = TRUE)
%R options(jsonlite.quiet = TRUE)
%R suppressMessages(library(tidyverse))
%R suppressMessages(library(lubridate))
%R suppressMessages(library(jsonlite))
%R suppressMessages(options(dplyr.summarise.inform = FALSE))


     |████████████████████████████████| 25.3 MB 1.4 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


dplyr.summarise.inform,[RTYPES.NILSXP]


In [38]:
#hide
%%R

#Grab wormhole bridge query from Flipside
df_wh = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/d0c057eb-cbe7-49e4-8f5a-77b3672105e6/data/latest', simplifyDataFrame = TRUE)

#fix the column names
names(df_wh)<-tolower(names(df_wh))

#Change the date to date format
df_wh$block_timestamp <- parse_datetime(df_wh$block_timestamp)

df_wh <- df_wh %>% rename(bridge_chain_id = wormhole_chain_id)


#join chain ids
chains <- tibble(bridge_chain_id = c(0,1,2,3,4,5,6),
                 bridge_chain = c('Unknown','Solana','Ethereum','Terra','BSC','Polygon','Harmony'))
df <- df_wh %>% left_join(chains, by = "bridge_chain_id")

#create a date field 
df$date <- floor_date(df$block_timestamp, unit = 'day')

# clip by date to remove the last part date
#df <- df %>% filter(date < '2021-11-17')

#Grab the token labels
labels <- read_csv("https://github.com/scottincrypto/analytics/raw/master/data/wormhole_bridge_assets.csv", show_col_types = FALSE)

#join the token lables
df <- df %>%
  left_join(labels, by = "denom")

tx_by_day <- df %>%
  group_by(date) %>%
  summarise(qty = sum(amount),
            tx_count = n(),
            av_tx_size = qty / tx_count)
  
tx_by_day_excl_network_acc <- df %>%
  filter(user != 'terra1dtzfwgzt8xa70zkm8gqzwz0n4zrqtngdpqejx5') %>%
  group_by(date) %>%
  summarise(qty = sum(amount),
            tx_count = n(),
            av_tx_size = qty / tx_count)

  
#unique wallets 
unique_wallets <- df %>% 
  filter(user != 'terra1dtzfwgzt8xa70zkm8gqzwz0n4zrqtngdpqejx5') %>%
  group_by(user) %>% 
  summarise(first_date = min(date)) %>%
  ungroup() %>%
  group_by(first_date) %>% 
  summarise(unique_wallets = n()) %>%
  mutate(cum_unique = cumsum(unique_wallets),
         growth_rate = unique_wallets / cum_unique * 100)
  
#split by chain
tx_by_dest_chain <- df %>%
  filter(user != 'terra1dtzfwgzt8xa70zkm8gqzwz0n4zrqtngdpqejx5') %>%
  group_by(bridge_chain, date) %>%
  summarise(qty = sum(amount),
            tx_count = n(),
            av_tx_size = qty / tx_count)
  

tx_by_wallet <- df %>%
  group_by(user) %>%
  summarise(qty = sum(amount),
            tx_count = n(),
            av_tx_size = qty / tx_count)  
  
sol_by_day_of_week <- tx_by_dest_chain %>%
  filter(bridge_chain == 'Solana') %>% 
  filter(date != ymd('2021-11-04')) %>%
  mutate(weekday = weekdays(date, abbreviate = F),
         sort = wday(date)) %>%
  group_by(weekday, sort) %>%
  summarise(av_qty = mean(qty)) %>%
  arrange(sort)

# Sending UST into the Wormhole

The Wormhole Bridge went live on the Terra network at the end of September 2021.  Wormhole is a cross chain network which allows traffic between blockchains with different consensus mechanisms.  This is done in a decentralised manner, avoiding the need for the centralised liquidity pools which power many existing bridges.  When the Wormhole bridge connected to Terra, it allowed connectivity between Terra, Solana, Ethereum, Polygon and Binance Smart Chain.  This was a huge step forward for a multi-chain world, as Terra and Solana are very different technologies to the other three EVM compatible chains.

Initial use of the Wormhole bridge to & from Terra was light but is steadily increasing.  As was outlined in [this post,](https://scottincrypto.github.io/analytics/terra/wormhole/2021/11/17/A-Flight-Through-a-Terra-Wormhole.html) most of the traffic passing over the Wormhole Bridge to & from Terra is UST - the decentralised stablecoin native to Terra.  This post will look more deeply at UST being sent from Terra over the Wormhole Bridge.

# How much UST is being sent?

The graph below shows the amount of UST sent from the Terra network since the Wormhole Bridge was attached.  The graph is a little curious - there is a background rate of a million or so UST per day, puncuated by some very large volume days.  This is worth some exploration.

In [ ]:
#hide_input
#UST Sent via Wormhole
df_p = %R tx_by_day
fig = px.bar(df_p
             , x = "date"
             , y = "qty"
             , labels=dict(date="Date", qty="UST Sent")
             , title= "UST Sent via Wormhole"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='UST')
fig.update_xaxes(title_text=None)
fig.show()

## Investigating the big sends

The table below shows the top 10 senders of UST over the Wormhole Bridge in the time period of the chart above.  Notice the top user has sent over $124m UST in only 12 transactions.  The next largest user has sent 84m UST, but over many more transactions.

In investigating user our top wallet, terra..ejx5, we identified a single transaction which was 42m UST in size.  View this on [finder.terra.money here](https://finder.terra.money/mainnet/tx/C1213BE51F64E9F320C0D39B0AAF36C289CF586BE2496E80463B95BED87CB22D).

We know this transaction went to Ethereum (recipient chain 2) but the recipient address is obfuscated in the transaction so we can't easily identify it.  By looking for transaction on the Ethereum Wormhole Bridge contract at around the time of this transaction, we found this one:  [https://etherscan.io/tx/0xb827b16a9125dcbeffa839bd5be65bd35690b176e0f868abe4fa5afd31d18f2c](https://etherscan.io/tx/0xb827b16a9125dcbeffa839bd5be65bd35690b176e0f868abe4fa5afd31d18f2c), which shows our 42m UST, now a Wormhole wrapped version of UST, being claimed at the other end.  Further transactions on this wallet identify that the 42m UST was [deposited as liquidity](https://etherscan.io/tx/0xf3d25987e15237905c7efa87332a50ce83cf8dc704b00804551bdcea763c8bcb) into the Shuttle/Wormhole Pool.  This Pool on the Ethereum network serves to exchange wrapped UST tokens from the legacy Shuttle bridge for Wormhole wrapped UST ready for use on the new bridge.  This pool is described in the [Wormhole documentation](https://medium.com/terra-money/wormhole-v2-for-terra-the-ui-walkthrough-595ca6649ae8) and is not a for-profit dex pool.

Our conclusion is that our big sending address, terra..ejx5, belongs to the one of the Terra/Wormhole teams and is being used for network operations to stand up the bridge.  Since we are looking at usage, it's worthwhile to exclude this address from any further analysis as it doesn't represent organic usage of the bridge.


In [3]:
#hide_input
%R tx_by_wallet %>% arrange(desc(qty)) %>% head(10)

,user,qty,tx_count,av_tx_size
1,terra1dtzfwgzt8xa70zkm8gqzwz0n4zrqtngdpqejx5,1.240186e+08,12,1.033488e+07
2,terra1zhgnqk9gq295qtqqa9j5shmm7lkp9e66m937jr,8.420261e+06,90,9.355846e+04
3,terra1nckkt3jx9xna820c4zjfyedz97vel0g6s8sh35,6.888655e+06,29,2.375398e+05
4,terra1qggut0tm799s0sy65cnzdela3yj2rfamd2sssf,5.000000e+06,1,5.000000e+06
5,terra1mr8j3d3gc56l97nstl2w5q9p4hehrfr82hy8yu,4.999970e+06,5,9.999940e+05
6,terra1cr8gvqj86duhgfpvjs5x237v4wfsf0qykacsjx,2.123608e+06,5,4.247216e+05
7,terra1ajz9k7f3wesrvhdyhqgcedh4vxfzlng0aq65g7,1.969082e+06,20,9.845410e+04
8,terra1ejus678gnw9fzxhujvrm8vhv9xykvktk8560cl,1.730996e+06,6,2.884993e+05
9,terra18zfccrvysexp8gjwjy72x5xmurxhhftl5ax6f0,1.712820e+06,6,2.854700e+05
10,terra1pvt0wsjs7mk4nq8hzeznce7d782qk6cptksp7z,1.596270e+06,6,2.660450e+05


# How much UST is being sent - revisited

The graph below shows the amount of UST being sent per day from the Terra network via the Wormhole bridge, this time excluding the network operations account we identified earlier.  Once the bridge ramped up for the first week, it has settled into a range of 400k -> 7m per day, with a typical day being around 1.5m UST sent to other networks.

In [4]:
#hide_input
#UST Sent via Wormhole expluding the network liquidity account
df_p = %R tx_by_day_excl_network_acc 
fig = px.bar(df_p
             , x = "date"
             , y = "qty"
             , labels=dict(date="Date", qty="UST Sent")
             , title= "UST Sent via Wormhole, network ops count excluded"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='UST')
fig.update_xaxes(title_text=None)
fig.show()

# Where is the UST going?

As mentioned in the intro, the Wormhole bridge connects Terra to 4 other chains - Ethereum, Solana, Polygon and Binance Smart Chain (BSC).  The table below shows how much UST has gone to each chain since the bridge began (excluding the network ops account).  We can see that most of the UST sent has gone to Solana - 57m UST, around 83% of all the UST traffic.  Ethereum is next largest with 8m UST or 12%, and Polygon & BSC having comparatively small amounts.

In [11]:
#hide_input
%R tx_by_dest_chain %>% group_by (bridge_chain) %>% summarise(qty_sent = sum(qty)) %>% mutate(percentage = qty_sent / sum(qty_sent)*100 )%>% arrange(desc(qty_sent))

,bridge_chain,qty_sent,percentage
1,Solana,5.788127e+07,82.619677
2,Ethereum,8.294427e+06,11.839458
3,Polygon,2.215865e+06,3.162924
4,BSC,1.665926e+06,2.377941


The graph below shows the UST sent by destination chain plotted daily, excluding the network ops account.  We see a few large transactions dominating BSC, Ethereum and Polygon, but the network traffic over Solana is both larger and more consistent than the EVM chains.  It appears that sending UST to Solana is a use case that the market was looking for, and has been met with the Terra->Solana Wormhole bridge.

In [12]:
#hide_input
# User Counts
df_p = %R tx_by_dest_chain %>% select(-tx_count) %>% select (-av_tx_size) %>% pivot_wider(names_from = bridge_chain, values_from = qty, values_fill = 0) %>% arrange(date)

fig = make_subplots(rows=2, cols=2, subplot_titles=("BSC", "Ethereum", "Polygon", "Solana"))
fig.append_trace(go.Bar(x=df_p["date"], y=df_p["BSC"], name="UST"), row=1, col=1)
fig.append_trace(go.Bar(x=df_p["date"], y=df_p["Ethereum"], name="UST"), row=2, col=1)
fig.append_trace(go.Bar(x=df_p["date"], y=df_p["Polygon"], name="UST"), row=1, col=2)
fig.append_trace(go.Bar(x=df_p["date"], y=df_p["Solana"], name="UST"), row=2, col=2)
fig.update_layout(width=800, height=800/1.618, title_text="UST Sent by Destionation Chain")
fig.update_layout(template="simple_white", showlegend=False)
fig.update_yaxes(title_text='Value Bridged UST', row=1, col=1)
fig.update_yaxes(title_text='Value Bridged UST', row=2, col=1)
fig.show()

# Estimating User Count
User count is challenging in blockchain environments as people may operate multiple wallets.  The fallback for this problem is to use unique wallet addresses, which gives us an upper bound on the number of users.  For wallets sending UST via the Wormhole bridge, we get the following profile of unique accounts over time.  At the time of publication, just under 2000 unique wallets were recorded sending UST over Wormhole

In [13]:
#hide_input
#cumulative wallets
df_p = %R unique_wallets
fig = px.bar(df_p
             , x = "first_date"
             , y = "cum_unique"
             , labels=dict(date="Date", av_tx_size="Av Tx Size")
             , title= "Unique Wallets Sending UST over Wormhole Bridge"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='Wallet Count')
fig.update_xaxes(title_text=None)
fig.show()

The unique wallet growth rate is consistent but steady, ranging from 1.5% -> 3% daily after some rapid initial growth

In [14]:
#hide_input
#cumulative wallets growth rate
df_p = %R unique_wallets
fig = px.bar(df_p
             , x = "first_date"
             , y = "growth_rate"
             , labels=dict(date="Date", av_tx_size="Av Tx Size")
             , title= "Growth rate of unique wallets sending UST over Wormhole Bridge"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='Growth Rate')
fig.update_xaxes(title_text=None)
fig.show()

# Transaction Size

The charts below show the average transaction sizes of UST sent to each of the target chains, excluding the network ops address. Each chain has a small number of large transactions which skew the data on those days.  Outside of these, we see relatively small values bridged to BSC.  This makes sense in the context that BSC has very low transaction fees, and bridging a few hundred UST is still relatively efficient. This contrasts with Ethereum - the lower bound of the average transaction is closer to 2-3000 UST.  This shows that users are sensitive to the Ethereum gas fees associated with claiming the tokens on the other side of the bridge which will typically be on the order of 100-200 USD.

Polygon is an outlier when compared with BSC and Ethereum.  With low transaction fees, it might be expected that there are small transaction sizes sent to Polygon in line with BSC.  Instead we see that the typical day has transaction sizes in the 10-20k UST range.  This can be explained by a relatively small number of users doing large transactions, indicating that retail users haven't discovered this pathway to Polygon just yet.

Solana transaction sizes stand out compared to the other chains.  They are more consistent, ranging from 5k UST on a slow day to 30-60k on a bigger day.  There are no days with zero or very low transaction size, supporting the higher usage shown above.

In [15]:
#hide_input
# Tx size
df_p = %R tx_by_dest_chain %>% select(-tx_count) %>% select (-qty) %>% pivot_wider(names_from = bridge_chain, values_from = av_tx_size, values_fill = 0) %>% arrange(date)

fig = make_subplots(rows=2, cols=2, subplot_titles=("BSC", "Ethereum", "Polygon", "Solana"))
fig.append_trace(go.Bar(x=df_p["date"], y=df_p["BSC"], name="UST"), row=1, col=1)
fig.append_trace(go.Bar(x=df_p["date"], y=df_p["Ethereum"], name="UST"), row=2, col=1)
fig.append_trace(go.Bar(x=df_p["date"], y=df_p["Polygon"], name="UST"), row=1, col=2)
fig.append_trace(go.Bar(x=df_p["date"], y=df_p["Solana"], name="UST"), row=2, col=2)
fig.update_layout(width=800, height=800/1.618, title_text="Transaction Size by Destionation Chain")
fig.update_layout(template="simple_white", showlegend=False)
fig.update_yaxes(title_text='Tx Size UST', row=1, col=1)
fig.update_yaxes(title_text='Tx Size UST', row=2, col=1)
fig.show()

# What's going on with Solana?

We have seen above that the volume and patterns of UST sent to Solana is different to the three EVM chains (BSC, Ethereum, Polygon).  The Solana data is shown in the chart below (excluding network ops wallet).  There appears to be a cyclical component to the usage, possibly based on a weekday/weekend cycle.

In [21]:
#hide_input
#UST Sent via Wormhole expluding the network liquidity account
df_p = %R tx_by_dest_chain %>% filter(bridge_chain == 'Solana')
fig = px.bar(df_p
             , x = "date"
             , y = "qty"
             , labels=dict(date="Date", qty="UST Sent")
             , title= "UST Sent to Solana via Wormhole"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='UST')
fig.update_xaxes(title_text=None)
fig.show()

The chart below shows the average volume of UST sent to Solana from Terra by day of the week.  The very large day on the 4th of November was excluded as an outlier, as were the network operations transactions.  There seems to be an influence based on day of the week - Monday sees transaction volumes of nearly 2m UST, and Saturdays at 0.6m UST as the low point.  At face value this doesn't quite align with a normal work week.  The timestamps, however are in UTC.  APAC markets are ahead of UTC - Korea, a country known to use the Terra network heavily via the Chai payments network, is UTC+9 hours.  The chart below aligns with a timezone such as this, with activity on Sunday night UTC being Monday morning in Asia.  We can draw two conclusions from this:

-  Bridging to Solana is being done for work (or at work) as more transactions occur on weekdays as opposed to weekends
- It is likely that this activity is occurring in APAC markets due to the observed timezone offest

In [40]:
#hide_input
#UST Sent via Wormhole expluding the network liquidity account
df_p = %R sol_by_day_of_week
fig = px.bar(df_p
             , x = "weekday"
             , y = "av_qty"
             , labels=dict(date="Date", qty="UST Sent")
             , title= "UST Sent to Solana via Wormhole"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='UST')
fig.update_xaxes(title_text=None)
fig.show()

# Conclusions

The Wormhole Network is a new cross-chain bridge network, linking blockchains with different consensus mechanisms in a decentralised manner.  The connection of Wormhole into the Terra network has unleashed another avenue for UST to take over the world.  UST is the most commonly sent token sent via the Wormhole bridge from Terra, with typical volumes of 1.5m UST per day being sent.  Most of the volume is going to Solana, with only sporadic transaction volume going to the three EVM chains - Ethereum, BSC and Polygon.  Unique wallet counts have been growing steadily, at daily rates of 1.5-3%.  There are around 2000 unique wallets so far which have sent UST via this bridge.  Transaction sizes vary by destination, with some evidence that transaction fees may impact this size.  Finally, we saw that UST sent to Solana appeared to show commercial use, potentially in an APAC timezone due to the timing of the transaction volume.

All data was sourced from the curated on-chain data tables at [Flipside Crypto](https://flipsidecrypto.com/)